In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('/Users/lucastucker/REU-2023/archive/mnist_train.csv')
data = np.array(data)
np.random.shuffle(data)

In [3]:
data.shape

(60000, 785)

In [6]:
m, n = data.shape
test_set = data[0: 1000].T
Y_test = test_set[0]
X_test = test_set[1: n]

'We now have a 1 x 1000 Y_test and 784 x 1000 X_test for later'

In [51]:
train_set = data[1000: m].T
Y_train = train_set[0]
X_train = train_set[1: n] / 255 # MUST scale data to [0,1] range!

'Create weight and bias matrices for a single hidden layer NN with 10 nodes'

In [19]:
def init_params(X, Y):
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

In [34]:
def relu(Z):
    return np.maximum(0, Z)

In [35]:
def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

In [41]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = relu(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [23]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [44]:
def relu_deriv(Z):
    return Z > 0

In [47]:
def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * relu_deriv(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1)
    return dW1, db1, dW2, db2

In [39]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, rate):
    W2 = W2 - rate * dW2
    W1 = W1 - rate * dW1
    b2 = b2 - rate * db2
    b1 = b1 - rate * db1
    return W1, b1, W2, b2

In [28]:
def get_accuracy(preds, Y):
    return np.sum(preds == Y) / Y.size

In [42]:
def gradient_descent(X, Y, iterations, rate):
    W1, b1, W2, b2 = init_params(X, Y)
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, rate)
        if i % 10 == 0:
            print("Epoch: ", i)
            print(get_accuracy(np.argmax(A2, axis = 0), Y))
    return W1, b1, W2, b2

In [55]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.10)


Epoch:  0
0.13442372881355932
Epoch:  10
0.19298305084745762
Epoch:  20
0.2860169491525424
Epoch:  30
0.36247457627118645
Epoch:  40
0.42450847457627117
Epoch:  50
0.4844915254237288
Epoch:  60
0.535542372881356
Epoch:  70
0.5787796610169491
Epoch:  80
0.617457627118644
Epoch:  90
0.6476440677966102
Epoch:  100
0.6719491525423729
Epoch:  110
0.6919830508474576
Epoch:  120
0.7087457627118644
Epoch:  130
0.7227457627118644
Epoch:  140
0.7356101694915255
Epoch:  150
0.7465423728813559
Epoch:  160
0.7553220338983051
Epoch:  170
0.7634237288135594
Epoch:  180
0.7703559322033898
Epoch:  190
0.777728813559322
Epoch:  200
0.7840338983050847
Epoch:  210
0.7895593220338983
Epoch:  220
0.794542372881356
Epoch:  230
0.7990169491525424
Epoch:  240
0.8033728813559322
Epoch:  250
0.8076610169491525
Epoch:  260
0.8114745762711865
Epoch:  270
0.8149661016949152
Epoch:  280
0.8182372881355933
Epoch:  290
0.8210169491525424
Epoch:  300
0.8234745762711865
Epoch:  310
0.8258813559322034
Epoch:  320
0.82876

In [59]:
X = X_test
Y = Y_test
Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
get_accuracy(np.argmax(A2, axis = 0), Y)

/var/folders/33/zckmdctn235gr36g69vrq8sc0000gn/T/ipykernel_79316/2645878079.py:2: RuntimeWarning: overflow encountered in exp
  return np.exp(Z) / sum(np.exp(Z))
/var/folders/33/zckmdctn235gr36g69vrq8sc0000gn/T/ipykernel_79316/2645878079.py:2: RuntimeWarning: invalid value encountered in divide
  return np.exp(Z) / sum(np.exp(Z))


0.673